# comercio_ext_indices.tb_referencia_ncm_cuci
> ### Origem — `bronze/autoloader/landingbeca2026jan/balancacomercial/NCM_CUCI_delta`
## 📌 Descrição do arquivo

Classificação **CUCI** hierárquica (Seção → Divisão → Grupo → Subgrupo → Item).

|Coluna|Descrição técnica|Interpretação humana|
|---|---|---|
|`CO_CUCI_ITEM`|Código do item CUCI|Nível mais detalhado|
|`NO_CUCI_ITEM`|Nome do item|Descrição do item|
|`CO_CUCI_SUB`|Código do subgrupo|Nível acima do item|
|`NO_CUCI_SUB`|Nome do subgrupo|Descrição do subgrupo|
|`CO_CUCI_GRUPO`|Código do grupo|Agrupamento intermediário|
|`NO_CUCI_GRUPO`|Nome do grupo|Descrição do grupo|
|`CO_CUCI_DIVISAO`|Código da divisão|Agrupamento amplo|
|`NO_CUCI_DIVISAO`|Nome da divisão|Descrição da divisão|
|`CO_CUCI_SEC`|Código da seção|Macrogrupo CUCI|
|`NO_CUCI_SEC`|Nome da seção|Macrogrupo|

## Configurações
> #### **imports**
> #### **get files**
> #### **schema**

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import types as T
from delta.tables import DeltaTable
jvm = spark._jvm
FileSystem = jvm.org.apache.hadoop.fs.FileSystem
Path = jvm.org.apache.hadoop.fs.Path
fs = FileSystem.get(spark._jsc.hadoopConfiguration())

In [0]:
bronzePath = "/mnt/bronze/autoloader/landingbeca2026jan/balancacomercial/ncm_cuci/"
silverPath = "/mnt/silver/landingbeca2026jan/comercio_ext_indices/tb_referencia_ncm_cuci/"
silverTable = "silver_comercio_ext_indices.tb_referencia_ncm_cuci"

In [0]:
silverSchema = T.StructType([
    T.StructField("CO_CUCI_ITEM",   T.StringType(),   False),
    T.StructField("NO_CUCI_ITEM",   T.StringType(),   False),
    T.StructField("CO_CUCI_SUB",    T.StringType(),   True ),
    T.StructField("NO_CUCI_SUB",    T.StringType(),   True ),
    T.StructField("CO_CUCI_GRUPO",  T.StringType(),   True ),
    T.StructField("NO_CUCI_GRUPO",  T.StringType(),   True ),
    T.StructField("CO_CUCI_DIVISAO",T.StringType(),   True ),
    T.StructField("NO_CUCI_DIVISAO",T.StringType(),   True ),
    T.StructField("CO_CUCI_SEC",    T.StringType(),   True ),
    T.StructField("NO_CUCI_SEC",    T.StringType(),   True ),
    T.StructField("TS_REF",         T.TimestampType(),False),
    T.StructField("NM_ORIGEM",      T.StringType(),   False),
])

## Extração
> #### **saprk.read**

In [0]:
df_bronze = spark.read.format("delta").load(bronzePath)

##Normalização
> #### **datatype**

In [0]:
def up(s): return F.upper(F.trim(F.col(s).cast(T.StringType())))

df_norm = (
    df_bronze
    .withColumn("CO_CUCI_ITEM", up("CO_CUCI_ITEM"))
    .withColumn("NO_CUCI_ITEM", F.col("NO_CUCI_ITEM").cast(T.StringType()))
    .withColumn("CO_CUCI_SUB", up("CO_CUCI_SUB"))
    .withColumn("NO_CUCI_SUB", F.col("NO_CUCI_SUB").cast(T.StringType()))
    .withColumn("CO_CUCI_GRUPO", up("CO_CUCI_GRUPO"))
    .withColumn("NO_CUCI_GRUPO", F.col("NO_CUCI_GRUPO").cast(T.StringType()))
    .withColumn("CO_CUCI_DIVISAO", up("CO_CUCI_DIVISAO"))
    .withColumn("NO_CUCI_DIVISAO", F.col("NO_CUCI_DIVISAO").cast(T.StringType()))
    .withColumn("CO_CUCI_SEC", up("CO_CUCI_SEC"))
    .withColumn("NO_CUCI_SEC", F.col("NO_CUCI_SEC").cast(T.StringType()))
    .withColumn("TS_REF", F.current_timestamp())
    .withColumn("NM_ORIGEM", F.lit("/landingbeca2026jan/balancacomercial/ncm_cuci"))
)

##Validações
> #### **data quality**
> #### **deduplicação**
> #### **schema fit**

In [0]:
df_valid = df_norm.filter(F.col("CO_CUCI_ITEM").isNotNull())

In [0]:
df_dedup = df_valid.dropDuplicates(["CO_CUCI_ITEM"])

In [0]:
df_silver = df_dedup.select(
    "CO_CUCI_ITEM","NO_CUCI_ITEM","CO_CUCI_SUB","NO_CUCI_SUB",
    "CO_CUCI_GRUPO","NO_CUCI_GRUPO","CO_CUCI_DIVISAO","NO_CUCI_DIVISAO",
    "CO_CUCI_SEC","NO_CUCI_SEC","TS_REF","NM_ORIGEM"
)

##Carga
> #### **merge**

In [0]:
delta_target = DeltaTable.forName(spark, "silver_comercio_ext_indices.tb_referencia_ncm_cuci")

merge_condition = """
  t.CO_CUCI_ITEM   = s.CO_CUCI_ITEM
"""

(delta_target.alias("t")
    .merge(df_silver.alias("s"), merge_condition)
    .whenMatchedUpdate(set={
        "t.NO_CUCI_ITEM": "s.NO_CUCI_ITEM",
        "t.CO_CUCI_SUB": "s.CO_CUCI_SUB",
        "t.NO_CUCI_SUB": "s.NO_CUCI_SUB",
        "t.CO_CUCI_GRUPO": "s.CO_CUCI_GRUPO",
        "t.NO_CUCI_GRUPO": "s.NO_CUCI_GRUPO",
        "t.CO_CUCI_DIVISAO": "s.CO_CUCI_DIVISAO",
        "t.NO_CUCI_DIVISAO": "s.NO_CUCI_DIVISAO",
        "t.CO_CUCI_SEC": "s.CO_CUCI_SEC",
        "t.NO_CUCI_SEC": "s.NO_CUCI_SEC",
        "t.TS_REF": "s.TS_REF",
        "t.NM_ORIGEM": "s.NM_ORIGEM"
    })
    .whenNotMatchedInsertAll()
    .execute()
)